In [213]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.autograd import Variable

In [214]:
dataset = pd.read_csv('/content/drive/My Drive/toy/Churn_Modelling.csv')

In [215]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [216]:
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [217]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [218]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [219]:
# Normalization
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [220]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [221]:
y_train

array([0, 0, 0, ..., 0, 0, 1])

In [222]:
y = torch.from_numpy(y_train)
y.shape

torch.Size([8000])

In [223]:
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.fc1 = nn.Linear(12, 512)           
#         self.fc2 = nn.Linear(512, 512)          
#         self.fc3 = nn.Linear(512, 2)
#         self.dropout = nn.Dropout(0.2)
        
#     def forward(self, x):
#         x = F.relu(self.fc1(x))
#         x = self.dropout(x)
#         x = F.relu(self.fc2(x))
#         x = self.dropout(x)
#         x = self.fc3(x)
#         return x
# model = Net()
# model

In [224]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(12,512)            # obviamente tem que bater com as features, e a input
        self.fc2 = nn.Linear(512, 2)            # 512 escolhemos aleatoriamente, sao as hidden features
        
    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = self.fc2(x)
        x = F.softmax(x, dim=1)
        return x

model = Net()
model

# Metodo basico:
# model = nn.Sequential(
#             nn.Linear(12, 512),
#             nn.Tanh(),
#             nn.Linear(512, 2),
#             nn.Softmax(dim=1))

Net(
  (fc1): Linear(in_features=12, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)

In [225]:
[param.shape for param in model.parameters()]

[torch.Size([512, 12]),
 torch.Size([512]),
 torch.Size([2, 512]),
 torch.Size([2])]

In [226]:
for name, param in model.named_parameters():
  print(name, param.shape)

fc1.weight torch.Size([512, 12])
fc1.bias torch.Size([512])
fc2.weight torch.Size([2, 512])
fc2.bias torch.Size([2])


In [236]:
batch_no * 64

8000

In [227]:
loss_fn = nn.CrossEntropyLoss()                       # Lembrar que ao menos 2 outputs, para multilabel. Could use a BCEloss para binary.
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

batch_size = 64
n_epochs = 200
batch_no = len(X_train) // batch_size                 # 125. 125 * 64 = 8000

train_loss = 0
train_loss_min = np.Inf
for epoch in range(n_epochs):
    for i in range(batch_no):
        start = i * batch_size
        end = start + batch_size
        x_var = Variable(torch.FloatTensor(X_train[start:end]))
        y_var = Variable(torch.LongTensor(y_train[start:end])) 
        
        # output = model(x_var)
        # loss = loss_fn(output,y_var)


        optimizer.zero_grad()
        output = model(x_var)
        loss = loss_fn(output,y_var)
        loss.backward()
        optimizer.step()
        
        values, labels = torch.max(output, 1)
        num_right = np.sum(labels.data.numpy() == y_train[start:end])
        train_loss += loss.item() * batch_size
    
    train_loss = train_loss / len(X_train)
    if train_loss <= train_loss_min:
        print("Validation loss decreased ({:6f} ===> {:6f}). Saving the model...".format(train_loss_min,train_loss))
        torch.save(model.state_dict(), "model.pt")
        train_loss_min = train_loss
    
    if epoch % 200 == 0:
        print('')
        print("Epoch: {} \tTrain Loss: {} \tTrain Accuracy: {}".format(epoch + 1, train_loss,num_right / len(y_train[start:end])))
print('Training Ended! ')

Validation loss decreased (   inf ===> 0.570849). Saving the model...

Epoch: 1 	Train Loss: 0.570849015712738 	Train Accuracy: 0.84375
Validation loss decreased (0.570849 ===> 0.516916). Saving the model...
Validation loss decreased (0.516916 ===> 0.507635). Saving the model...
Validation loss decreased (0.507635 ===> 0.503356). Saving the model...
Validation loss decreased (0.503356 ===> 0.500733). Saving the model...
Validation loss decreased (0.500733 ===> 0.498886). Saving the model...
Validation loss decreased (0.498886 ===> 0.497474). Saving the model...
Validation loss decreased (0.497474 ===> 0.496332). Saving the model...
Validation loss decreased (0.496332 ===> 0.495374). Saving the model...
Validation loss decreased (0.495374 ===> 0.494547). Saving the model...
Validation loss decreased (0.494547 ===> 0.493818). Saving the model...
Validation loss decreased (0.493818 ===> 0.493165). Saving the model...
Validation loss decreased (0.493165 ===> 0.492574). Saving the model...


In [228]:
X_test_var = Variable(torch.FloatTensor(X_test), requires_grad=False) 
with torch.no_grad():
    test_result = model(X_test_var)
values, labels = torch.max(test_result, dim=1)

In [229]:
test_result

tensor([[1.0000e+00, 7.5605e-07],
        [8.6843e-01, 1.3157e-01],
        [1.0000e+00, 9.8866e-07],
        ...,
        [9.8655e-01, 1.3450e-02],
        [1.0000e+00, 2.3970e-07],
        [9.8574e-01, 1.4256e-02]])

In [230]:
torch.max(test_result, 1)

torch.return_types.max(values=tensor([1.0000, 0.8684, 1.0000,  ..., 0.9865, 1.0000, 0.9857]), indices=tensor([0, 0, 0,  ..., 0, 0, 0]))

In [231]:
values

tensor([1.0000, 0.8684, 1.0000,  ..., 0.9865, 1.0000, 0.9857])

In [232]:
labels

tensor([0, 0, 0,  ..., 0, 0, 0])

In [233]:
from sklearn import metrics
metrics.accuracy_score(y_test, labels)

0.846

In [234]:
x_var = Variable(torch.FloatTensor(X_train[start:end]))
x_var.dtype

torch.float32